### Testing Playground for P0 Programs &ndash; Array Concatenation

In [ ]:
import nbimporter; nbimporter.options["only_defs"] = False
from P0 import compileString
from ST import printSymTab
import time

def runwasm(wasmfile):
    from IPython.core.display import display, Javascript
    display(Javascript("""
    const params = { 
        P0lib: { 
            write: i => this.append_stream({text: '' + i, name: 'stdout'}),
            writeln: () => this.append_stream({text: '\\n', name: 'stdout'}),
            read: () => window.prompt()
        }
    }

    fetch('""" + wasmfile + """') // asynchronously fetch file, return Response object
      .then(response => response.arrayBuffer()) // read the response to completion and stores it in an ArrayBuffer
      .then(code => WebAssembly.compile(code)) // compile (sharable) code.wasm
      .then(module => WebAssembly.instantiate(module, params)) // create an instance with memory
    // .then(instance => instance.exports.program()); // run the main program; not needed if start function specified
    """))

Write, compile and execute P0 programs for testing using the following steps:
1. Define a string constant containing a P0 program to be validated in the cell below.
2. Update the `compileString` parameter to accept the string constant in the cell below.
3. Select `Kernel > Restart & Run All` to view and execute the target WebAssembly program.
   <br>Alternatively, selecting `Cell > Run All` is sufficient to rerun if there are no code changes.

In [ ]:
# Concatenating array with single element array literal
test1 = """
program p
    var a: [1..3] → integer
    a := [3, 7, 11]
    a := a + [1]
    write(a[4])
"""

# Concatenating array with multiple element array literal
test2 = """
program p
    var a: [1..3] → integer
    a := [3, 7, 11]
    a := a + [1, 5, 9]
    write(a[4])
    write(a[5])
    write(a[6])
"""

# Concatenating array literal with array literal
test3 = """
program p
    var a: [1..3] → integer
    a := [3, 7, 11] + [1, 5, 9]
    write(a[4])
    write(a[5])
    write(a[6])
"""

# Update this line to run a specific testcase
compileString(test1, "test.wat")

In [ ]:
!cat -n "test.wat"

In [ ]:
!wat2wasm --enable-bulk-memory test.wat || rm test.wasm

In [ ]:
runwasm("test.wasm")

In [ ]:
time.sleep(1)

The above tests can also be set to run all at once using the following steps:
1. Define testcases using the following format: `(testcase, pass, result)`
   <br>`testcase` is a string such as the above containing a P0 program to be tested.
   <br>`pass` is a Boolean value set to `True` if the program should compile/run without raising an exception.
   <br>`result` is the expected output or exception message when the given program is compiled/run.
2. Set the value of `runall` to `True` below (set to `False` by default to prevent unnecessary runs).
3. Rerun the notebook as above, or run the below cells by selecting `Cell > Run All Below` from the below cell.

Note: the automated test runs should ideally be modified below to capture the output of the `runwasm` function and compare
<br>against `test[2]` to check for the expected result if the program runs successfully, but this has not yet been implemented.

In [ ]:
tests = [
    (test1, True, "1"),
    (test2, True, "1\n5\n9"),
    (test3, True, "1\n5\n9"),
]

In [ ]:
%%capture errors
runall = False
passing = 0
if runall:
    for index, test in enumerate(tests):
        try:
            compileString(test[0], "test.wat")
            !wat2wasm --enable-bulk-memory test.wat || rm test.wasm
            runwasm("test.wasm")
            # TODO: compare runwasm output to test[2]
            if test[1]: passing += 1
            else: print(f'Test #{index + 1} did not throw an exception')
        except Exception as e:
            if not test[1] and test[2] in str(e): passing += 1
            elif not test[1]: print(f'Test #{index + 1} threw an unexpected exception: {e}')
            else: print(f'Test #{index + 1} threw an unhandled exception: {e}')

In [ ]:
if runall:
    print(f'{passing} out of {len(tests)} tests pass')
    print(errors)
else:
    print("Skipped automated testing")